In [1]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


# Download Crashdata from 2017 to 2019

In [2]:
import os
import requests
import zipfile

url_list = [
    "https://www.opengeodata.nrw.de/produkte/transport_verkehr/unfallatlas/Unfallorte2017_EPSG25832_CSV.zip",
    "https://www.opengeodata.nrw.de/produkte/transport_verkehr/unfallatlas/Unfallorte2018_EPSG25832_CSV.zip",
    "https://www.opengeodata.nrw.de/produkte/transport_verkehr/unfallatlas/Unfallorte2019_EPSG25832_CSV.zip"
]

directory = "../data"
if not os.path.exists(directory):
    os.makedirs(directory)

for url in url_list:
    file_name = url.split("/")[-1]
    year = file_name[10:14]
    subdirectory = os.path.join(directory, f"Unfallorte{year}")
    if not os.path.exists(subdirectory):
        os.makedirs(subdirectory)
    
    file_path = os.path.join(subdirectory, file_name)
    
    response = requests.get(url)
    if response.status_code == 200:
        with open(file_path, 'wb') as file:
            file.write(response.content)
        print(f"File {file_path} downloaded successfully!")
        
        # Unzip the downloaded file
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            zip_ref.extractall(subdirectory)
        print(f"File {file_path} extracted successfully!")
    else:
        print(f"Failed to download {file_name}. Status code: {response.status_code}")

File ../data/Unfallorte2017/Unfallorte2017_EPSG25832_CSV.zip downloaded successfully!
File ../data/Unfallorte2017/Unfallorte2017_EPSG25832_CSV.zip extracted successfully!
File ../data/Unfallorte2018/Unfallorte2018_EPSG25832_CSV.zip downloaded successfully!
File ../data/Unfallorte2018/Unfallorte2018_EPSG25832_CSV.zip extracted successfully!
File ../data/Unfallorte2019/Unfallorte2019_EPSG25832_CSV.zip downloaded successfully!
File ../data/Unfallorte2019/Unfallorte2019_EPSG25832_CSV.zip extracted successfully!


# Download Weatherdata

In [3]:
!jv weatherdata.jv

[WeatherData] Overview:
	Blocks (7 blocks with 1 pipes):
	 -> WeatherExtractor (HttpExtractor)
		 -> WeatherTextFileInterpreter (TextFileInterpreter)
			 -> WeatherCSVInterpreter (CSVInterpreter)
				 -> WeatherHeaderWriter1 (CellWriter)
					 -> WeatherHeaderWriter2 (CellWriter)
						 -> WeatherTableInterpreter (TableInterpreter)
							 -> WeatherLoader (SQLiteLoader)



# Add Crashdata to data.sqlite

In [4]:
import os
import sqlite3
import csv
import configparser

# Specify the path to the data directory and the database file
data_directory = "../data"
database_file = os.path.join(data_directory, "data.sqlite")

# Connect to the SQLite database
conn = sqlite3.connect(database_file)
cursor = conn.cursor()

# Iterate through the folders in the data directory
for year in range(2017, 2020):
    folder_name = f"Unfallorte{year}"
    folder_path = os.path.join(data_directory, folder_name)
    csv_folder_path = os.path.join(folder_path, "csv")

    # Check if the folder exists
    if os.path.isdir(folder_path):
        table_name = f"Unfallorte{year}"

        # Read the schema.ini file to extract the column names
        schema_ini_path = os.path.join(csv_folder_path, "schema.ini")
        config = configparser.ConfigParser()
        config.read(schema_ini_path)

        # Get the column names from the schema.ini file using the table name as the key
        column_names = []
        for key, value in config[f"{table_name}_LinRef.txt"].items():
            if key.lower().startswith("col") and key.lower() != "colnameheader":
                column_name = value.split()[0]
                column_names.append(column_name)

        # Create a table with the folder name as the table name
        create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({', '.join(column_names)})"
        cursor.execute(create_table_query)
        print(f"Table '{table_name}' created successfully.")

        # Read the TXT file within the csv folder and insert data into the table
        txt_file_path = os.path.join(csv_folder_path, f"{table_name}_LinRef.txt")
        with open(txt_file_path, newline='') as file:
            csv_reader = csv.reader(file, delimiter=';')
            next(csv_reader)  # Skip the header row if present
            for row in csv_reader:
                insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['?'] * len(column_names))})"
                cursor.execute(insert_query, row)

        print(f"Data from '{txt_file_path}' inserted into table '{table_name}'.")

# Commit the changes and close the database connection
conn.commit()
conn.close()


Table 'Unfallorte2017' created successfully.
Data from '../data/Unfallorte2017/csv/Unfallorte2017_LinRef.txt' inserted into table 'Unfallorte2017'.
Table 'Unfallorte2018' created successfully.
Data from '../data/Unfallorte2018/csv/Unfallorte2018_LinRef.txt' inserted into table 'Unfallorte2018'.
Table 'Unfallorte2019' created successfully.
Data from '../data/Unfallorte2019/csv/Unfallorte2019_LinRef.txt' inserted into table 'Unfallorte2019'.
